<a href="https://colab.research.google.com/github/JinFree/Thermo-and-Fluid-Engineering-Lab.1/blob/master/2020/Python_Elliptic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import shutil
import numpy as np

In [0]:
class Elliptic:
    enumerate(['Jacobi', 'PGS'])

    def __init__(self):
        self.Gx = 21
        self.Gy = 41
        self.GridSize = 0.05
        self.T0 = 100
        self.Ermax = 0.000001
        self.T = np.random.randn(self.Gy, self.Gx)
        self.Tnew = np.random.randn(self.Gy, self.Gx)
        self.scheme = 'error'
        self.Scheme_name = "Error"
        self.Dirname = "Error"
        self.iter = 0
        return

    def Scheme_Printer(self):
        print("Solve {0}".format(self.scheme))
        self.Scheme_name = "%s"%self.scheme
        return

    def Dir_Write(self):
        self.dirname = "Elliptic,{0}".format(self.Scheme_name)
        path = os.getcwd()
        self.dirname = os.path.join(path, self.dirname)
        if os.path.isdir(self.dirname):
            shutil.rmtree(self.dirname)  # 디렉토리가 존재하면 삭제하고 다시 계산# return # 디렉토리가 존재하면 덮어쓰기 #
        os.mkdir(self.dirname)
        return

    def Para_Write(self):
        filename = "{0}/{1}, iter = {2}.csv".format(self.dirname, self.Scheme_name, self.iter)
        file = open(filename, 'w')
        file.write("X,Y,Z,Temperature\n")
        for i in range(self.Gy):
            for j in range(self.Gx):
                data = "%3.3f,%3.3f,%3.3f,%3.3f\n" % \
                       (float(j*self.GridSize), float(i*self.GridSize), 0.0, self.T[i][j])
                file.write(data)
        file.close()
        return

    def Initial_Condition(self):
        self.T = np.zeros((self.Gy, self.Gx))
        self.Tnew = np.zeros((self.Gy, self.Gx))
        return

    def Boundary_Condition(self):
        for i in range(self.Gy):
            self.T[i][0] = 0
            self.Tnew[i][0] = 0
            self.T[i][self.Gx-1] = 0
            self.Tnew[i][self.Gx-1] = 0
        for i in range(self.Gx):
            self.T[self.Gy-1][i] = 0
            self.Tnew[self.Gy-1][i] = 0
            self.T[0][i] = self.T0
            self.Tnew[0][i] = self.T0
        return

    def CalErr(self):
        error = 0
        for i in range(1, self.Gy-1):
            for j in range(1, self.Gx-1):
                error += abs(self.Tnew[i][j] - self.T[i][j])
        return error

    def Memcpy(self):
        self.T = self.Tnew.copy()
        return

    def Jacobi(self):
        for i in range(1, self.Gy-1):
            for j in range(1, self.Gx-1):
                self.Tnew[i][j] = (self.T[i+1][j]+self.T[i-1][j]+self.T[i][j+1]+self.T[i][j-1])/4.0
        return

    def Jacobi_Solver(self):
        self.iter = 0
        error = 1.0
        while error > self.Ermax:
            self.Jacobi()
            error = self.CalErr()
            self.Memcpy()
            self.iter += 1
            print("\rIteration = %d"%self.iter, end="")
        return

    def PGS(self):
        for i in range(1, self.Gy-1):
            for j in range(1, self.Gx-1):
                self.Tnew[i][j] = (self.T[i+1][j]+self.Tnew[i-1][j]+self.T[i][j+1]+self.Tnew[i][j-1])/4.0
        return

    def PGS_Solver(self):
        self.iter = 0
        error = 1.0
        while error > self.Ermax:
            self.PGS()
            error = self.CalErr()
            self.Memcpy()
            self.iter += 1
            print("\rIteration = %d"%self.iter, end="")
        return

    def Main(self, scheme):
        self.scheme = scheme
        self.Scheme_Printer()
        self.Initial_Condition()
        self.Boundary_Condition()
        self.Dir_Write()
        if scheme == 'Jacobi':
            self.Jacobi_Solver()
        elif scheme == 'PGS':
            self.PGS_Solver()
        self.Para_Write()
        print()
        return

In [0]:
PyCompute = Elliptic()

In [8]:
PyCompute.Main('Jacobi')

Solve Jacobi
Iteration = 2268


In [9]:
PyCompute.Main('PGS')

Solve PGS
Iteration = 1170


In [0]:
!zip -r -qq Elliptic,Jacobi.zip Elliptic,Jacobi
!zip -r -qq Elliptic,PGS.zip Elliptic,PGS